In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pylab as pl

In [2]:
def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()

def linecount(file_name):
    with open(file_name, 'r') as foo:
        x = len(foo.readlines())
        return x

In [9]:
# Set system names and path to directory with .pdb files

pc=[0,11,22,33,44,50,37]
molec = [1000, 2000, 3000, 4000, 5000, 6500, 8000, 9000, 10000]
#pc=[11,22,33,44,50]
#molec = [3000]

samsfolder = "/Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/SAM_startfiles_v2_36x36/"
#samsfolder = "/Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/SAM_startfiles_v2_12x12/"

os.chdir(samsfolder)

In [ ]:
# Create .gro files from .pdb files
for m in pc:
    pdbfile = 'start%d.pdb'%(m, ) 
    grofile = 'start%d.gro'%(m, )
    print("gmx editconf -f %s -o %s" %(pdbfile, grofile, ))
    os.system("gmx editconf -f %s -o %s -c" %(pdbfile, grofile, ))   

In [10]:
# Print SAMs box sizes (last line of .gro file)
for m in pc:
    newfile = 'start%d.gro'%(m, )   
    last_line = open(newfile, "r").readlines()[-1]
    Line_len = len(last_line)
    print newfile, ":", last_line

start0.gro :   18.00000  15.58850   1.52200

start11.gro :   18.00000  15.58850   1.52200

start22.gro :   18.00000  15.58850   1.52200

start33.gro :   18.00000  15.58850   1.52200

start44.gro :   18.00000  15.58850   1.52200

start50.gro :   18.00000  15.58850   1.52200

start37.gro :   18.00000  15.58850   1.52200



In [20]:
# Use forward slash '/' at the end of the paths!!!

#folder water .gro files (and SAM .gro files)
workingdir = "/Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/DropsGroTop/"  #path to .gro files

# folder for storing shifted sams: can be deleted if everything works fine
unwantedfolder = "/Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/reshaping_files/" #path unwanted files

In [21]:
# Copy SAM .gro files to working directory (and print commands to check)
for m in pc:
    newfile = 'start%d.gro'%(m, )
    print("cp %s %s" %(newfile, workingdir+newfile, ))
    os.system("cp %s %s" %(newfile, workingdir+newfile, ))

cp start0.gro /Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/DropsGroTop/start0.gro
cp start11.gro /Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/DropsGroTop/start11.gro
cp start22.gro /Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/DropsGroTop/start22.gro
cp start33.gro /Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/DropsGroTop/start33.gro
cp start44.gro /Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/DropsGroTop/start44.gro
cp start50.gro /Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/DropsGroTop/start50.gro
cp start37.gro /Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/DropsGroTop/start37.gro


In [22]:
# New box size in z-direction
zboxnew=12.0

# sam sizes in z-direction (not box size!!)
sam={}

#sizes of simulation box of sams
xbox={}
ybox={}
zbox={}

mynumbers = []
mynumbers.append([float(n) for n in last_line.split()])

for i in pc:
    sam[i]= mynumbers[0][2]

    xbox[i]=mynumbers[0][0]
    ybox[i]=mynumbers[0][1]
    zbox[i]=mynumbers[0][2]

# water drop sizes in z-direction (they could also be read, look at script ExtendSam.ipynb)
water={}
water[216]=2.020
water[1000]=3.320
water[2000]=4.124
water[3000]=4.637
water[4000]=5.124
water[5000]=5.495
water[6500]=5.981 
water[7000]=6.126
water[8000]=6.395
water[9000]=6.665
water[10000]=6.895  

In [24]:
os.chdir(workingdir)
    
for i in pc:   
    
    # SAM .gro file names
    startsamfile='start%d.gro'%(i, )
    
    # Temporary SAM file names
    samfile='start'+str(i)+'_c.gro'
    
    # Center SAMs in box
    os.system("gmx editconf -f %s -o %s -c" %(startsamfile, samfile, ))
  
    for j in molec:        
        
        # Water (old) file names
        startwaterfile='NPT_water'+str(j)+'.gro'
        waterfile='NPT_water'+str(j)+'_c.gro'
        
        # New water files names
        newsystfile='NPT_sam'+str(i)+'_water'+str(j)+'_c.gro'
        endfile='NPT_sam'+str(i)+'_water'+str(j)+'.gro'
               
        # Set water box size equal to SAM box size
        os.system("gmx editconf -f %s -o %s -box %5.3f %5.3f %5.3f" %(startwaterfile, waterfile, xbox[i], ybox[i], zbox[i], ))
               
        # Move water drops up 
        os.system("gmx editconf -f %s -o %s -translate 0 0 %5.3f" %(waterfile, waterfile, (zbox[i]/2) +(water[j]/2) + 0.5, )) 
        
##### START THE MERGING PROCESS OF water drops + SAMs #################

        # Create the new file by openning and closing it
        creating = open(newsystfile, 'w+')
        creating.close()
        
        #linecount
        linewater=linecount(waterfile)
        linesam=linecount(samfile)
        
        lastline = open(samfile, "r").readlines()[-1]
        with open(newsystfile, 'a') as f1:
            with open(samfile, 'r') as g:
                l = 0
                for line in g:
                    if l < linesam-1: 
                        f1.write(line)
                    l = l+1   
            with open(waterfile, 'r') as h:
                l = 0
                for line in h:                        
                    if l > 1 and l < linewater-1:
                        f1.write(line)
                    l = l + 1
            f1.write(lastline)

             
        # Replace atom number
        atomssam = linesam-3
        atomswater = linewater-3
            
        replace_line(newsystfile, 1, str(atomssam+atomswater) + "\n")
        
        # Replace first line (title)
        replace_line(newsystfile, 0, "SAM "+str(i)+"% OH-coverage + water "+str(j)+ "\n")
        
        # Change box height (z-direction)
        os.system("gmx editconf -f %s -o %s -box %5.3f %5.3f %5.3f" %(newsystfile, endfile, xbox[i], ybox[i], zboxnew, ))
        
        # Translate system to bottom of box
        #os.system("gmx editconf -f %s -o %s -translate %8.5f %8.5f %8.5f" %(endfile, endfile, 0, 0,(zbox[i]-water[j]+0.5)/2, ))

gmx editconf -f start0.gro -o start0_c.gro -c
gmx editconf -f start11.gro -o start11_c.gro -c
gmx editconf -f start22.gro -o start22_c.gro -c
gmx editconf -f start33.gro -o start33_c.gro -c
gmx editconf -f start44.gro -o start44_c.gro -c
gmx editconf -f start50.gro -o start50_c.gro -c
gmx editconf -f start37.gro -o start37_c.gro -c


In [25]:
print("mv %s > %s"%(samfile,unwantedfolder))

mv start37_c.gro > /Users/eixeres/Dropbox/GitHub/LineTensionPackage/SYSTEM_CREATION/gromacs_files/reshaping_files/


In [26]:
# Delete/move not needed files

# Create folder for files not needed
os.system("mkdir %sdelete"% (str(unwantedfolder)))

os.system("rm \#*")

for i in pc:
    
    samfile='start'+str(i)+'_c.gro'
    os.system("mv %s  %s"%(samfile,unwantedfolder+'delete/'+samfile, ))
    
    for j in molec:
        
        newsystfile='NPT_sam'+str(i)+'_water'+str(j)+'_c.gro'
        os.system("mv %s  %s"% (newsystfile,unwantedfolder+'delete/'+newsystfile, ))       
        
for j in molec:
        
    waterfile='NPT_water'+str(j)+'_c.gro'
    os.system("mv %s  %s"% (waterfile,unwantedfolder+'delete/'+waterfile, ))

In [27]:
#Renaming and moving endfiles to endfolder

startfolder = workingdir
endfolder=workingdir + "NewVersion_v2/"

#os.system("mkdir %s"% (endfolder))
os.chdir(endfolder)

for i in pc:
    for j in molec:
        startname='NPT_sam'+str(i)+'_water'+str(j)+'.gro'
        endname='sam'+str(i)+'_water'+str(j)+'.gro'
        os.system("mv %s  %s"% (startfolder+startname,endfolder+endname, ))

In [28]:
# read position of bottom atom of SAM and save in dictionary zshift and translate system
line_num = 13 # some chains are longer then others: 12 or 13. Tested with "if-statement"
zshift = {}
for i in pc:
    for j in molec:
        endname='sam'+str(i)+'_water'+str(j)+'.gro'
        lines = open(endname, 'r').readlines()
        myline = lines[line_num]
        mynumbers = []
        mynumbers.append([str(n) for n in myline.split()])
        firstchain = str(mynumbers[0][0])
        #print len(firstchain), firstchain
        if firstchain == '1SAM' or firstchain == '1XOH':
            print 'chainlength = ', line_num
            print myline
            zshift[i,j] = float(mynumbers[0][5])
            print firstchain, zshift[i,j]
            os.system("gmx editconf -f %s -o %s -translate %8.5f %8.5f %8.5f" %(endname, endname, 0, 0,-zshift[i,j], ))
        else:
            print 'chainlength = ', line_num
            line_num = 12
            myline = lines[line_num]
            print myline
            mynumbers = []
            mynumbers.append([str(n) for n in myline.split()])
            zshift[i,j] = float(mynumbers[0][5])
            print str(mynumbers[0][0]), zshift[i,j]
            os.system("gmx editconf -f %s -o %s -translate %8.5f %8.5f %8.5f" %(endname, endname, 0, 0,-zshift[i,j], ))

chainlength =  13
    1SAM    CAB   11   0.172   0.534   4.916  0.0000  0.0000  0.0000

1SAM 4.916
chainlength =  12
    1SAM    CAB   11   0.172   0.534   4.442  0.0000  0.0000  0.0000

1SAM 4.442
chainlength =  12
    1SAM    CAB   11   0.173   0.534   4.065  0.0000  0.0000  0.0000

1SAM 4.065
chainlength =  12
    1SAM    CAB   11   0.173   0.534   3.696  0.0000  0.0000  0.0000

1SAM 3.696
chainlength =  12
    1SAM    CAB   11   0.173   0.534   3.396  0.0000  0.0000  0.0000

1SAM 3.396
chainlength =  12
    1SAM    CAB   11   0.173   0.534   2.986  0.0000  0.0000  0.0000

1SAM 2.986
chainlength =  12
    1SAM    CAB   11   0.172   0.534   2.656  0.0000  0.0000  0.0000

1SAM 2.656
chainlength =  12
    1SAM    CAB   11   0.173   0.534   2.440  0.0000  0.0000  0.0000

1SAM 2.44
chainlength =  12
    1SAM    CAB   11   0.173   0.534   2.256  0.0000  0.0000  0.0000

1SAM 2.256
chainlength =  12
    1SAM    CAB   11   0.175   0.538   4.914  0.0000  0.0000  0.0000

1SAM 4.914
chainlength

In [29]:
os.system("rm \#*")

0